In [322]:
import pandas as pd
from datetime import datetime
import openpyxl
import mysql.connector.connection
import dotenv
import os 

dotenv.load_dotenv()

True

CONFIGURACION

In [323]:
month = 3
year = 2025

                                                --- NO TOCAR ---                                                                    
                                                            

In [324]:
# Crear un objeto datetime con el mes y año (el día puede ser arbitrario, por ejemplo, 1)
date_obj = datetime(year, month, 1)  
date_str = date_obj.strftime("%m/%Y")

name_month = date_obj.strftime("%B")
file_name = f"{name_month} {year}.xlsx"
file_name

'March 2025.xlsx'

In [325]:
def conexion_db(host, user, password, database):
    try:
        cnx = mysql.connector.connect(host=host, user=user, password=password, database=database)
        print("✅ Conexion a base de datos establecida correctamente")
        return cnx
    except Exception as error:
        print(f"❌ Ocurrio un error al intentarse conectar a la base de datos\n \
              Error: {error}")
def query(cnx, query):
    try:
        cursor = cnx.cursor()
        cursor.execute(query)
        resultado = cursor.fetchall()
        if resultado is not None:
            print("✅ Se extrajeron los datos correctamente")
        else:
            print("❌ La consulta vino vacia")
        cursor.close()
        cnx.close()
        print("✅ Se cerro la conexion con base de datos")
        return resultado
    except Exception as error_query:
        print(f"❌ Ocurrio un error al intentar ejecutar la consulta\n \
              Error: {error_query}")


In [326]:
### Consutla extraccion de datos
# PAYROLL
payroll_extraction_query = f"""
SELECT
legajo,
documento, 
apellidos, 
nombres, 
equipo, 
fecha, 
codigo, 
descripcion, 
horas_programadas, 
horas_trabajadas,
usuario_neotel,
usuario_avaya,
campana, 
sub_campana, 
area, 
puesto
FROM payroll
WHERE MONTH(fecha) = {month} AND YEAR(fecha) = {year} ;""" 

columnas_payroll = ["legajo","documento", "apellidos", "nombres","equipo","fecha", "codigo", "descripcion", 
"horas_programadas","horas_trabajadas","usuario_neotel","usuario_avaya","campana", "sub_campana","area", "puesto"]

# NOMINA
nomina_extraction_query = f"""
SELECT legajo, documento, apellidos, nombres 
FROM asesores 
where fecha_baja is null or (month(fecha_baja) >= {month} and year(fecha_baja) = {year})  ;
"""
columnas_nomina = ["legajo","documento", "apellidos", "nombres"]

# Fechas_baja 
nomina_extraction_query_fechas = f"""
 select legajo, fecha_baja, fecha_alta 
 from asesores 
 where (month(fecha_baja) >= {month} and year(fecha_baja) = {year}) or
(month(fecha_alta) = {month} and year(fecha_alta) = {year})"""
# EXTRA
lista_columnas_vacias = ["Hs Extras", "Dep. Judicial", "Prepaga","Enlazados", "Ant. de Sueldos", "Adic. Ventas"]


In [327]:
# Consumimos payroll
cnx = conexion_db(os.getenv("DB_HOST"), os.getenv("DB_USER"), os.getenv("DB_PASS"), os.getenv("DB_NAME"))
resultado = query(cnx,payroll_extraction_query)
payroll = pd.DataFrame(resultado, columns=columnas_payroll)
payroll.head()

✅ Conexion a base de datos establecida correctamente
✅ Se extrajeron los datos correctamente
✅ Se cerro la conexion con base de datos


,legajo,documento,apellidos,nombres,equipo,fecha,codigo,descripcion,horas_programadas,horas_trabajadas,usuario_neotel,usuario_avaya,campana,sub_campana,area,puesto
0,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-03,DNL,Día No Laborable,8.0,0.0,NaN,NaN,None,None,None,None
1,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-04,DNL,Día No Laborable,8.0,0.0,NaN,NaN,None,None,None,None
2,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-05,V,Vacaciones,0.0,0.0,NaN,NaN,None,None,None,None
3,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-06,V,Vacaciones,0.0,0.0,NaN,NaN,None,None,None,None
4,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-07,V,Vacaciones,0.0,0.0,NaN,NaN,None,None,None,None


In [328]:
cnx = conexion_db(os.getenv("DB_HOST"), os.getenv("DB_USER"), os.getenv("DB_PASS"), os.getenv("DB_NAME"))
resultado = query(cnx,nomina_extraction_query)
nomina = pd.DataFrame(resultado, columns=columnas_nomina)
nomina.head()

✅ Conexion a base de datos establecida correctamente
✅ Se extrajeron los datos correctamente
✅ Se cerro la conexion con base de datos


,legajo,documento,apellidos,nombres
0,1317,18902640,CACERES PEREIRA,ANDRES FEDERICO
1,1600,19113210,RESTREPO ORDOSGOITIA,ARIANA ESTEFANNY
2,1601,19113447,RODRIGUEZ ALFONZO,NAIYELY HECNAY
3,884,19116652,APABLAZA GEREZ,ELIZABETH SABRINA
4,1229,23081765,PEREYRA,MARIA NORMA


In [329]:
cnx = conexion_db(os.getenv("DB_HOST"), os.getenv("DB_USER"), os.getenv("DB_PASS"), os.getenv("DB_NAME"))
resultado = query(cnx,nomina_extraction_query_fechas)
fechas_baja = pd.DataFrame(resultado, columns=["legajo", "fecha_baja", "fecha_alta"])
fechas_baja.head()

✅ Conexion a base de datos establecida correctamente
✅ Se extrajeron los datos correctamente
✅ Se cerro la conexion con base de datos


,legajo,fecha_baja,fecha_alta
0,1337,2025-03-14,2023-03-01
1,973,2025-03-25,2022-05-02
2,2086,None,2025-03-20
3,2087,None,2025-03-20
4,2089,None,2025-03-20


In [330]:
### Creacion de columnas en nomina que me realicen el conteo de dias dependiendo el n° de legajo
nomina["Basico"] = 30

# Contar cuántas veces aparece cada legajo con código "x" (D. Enfermedad)
enfermedad = payroll[payroll['codigo'] == 'CM'].groupby('legajo').size().reset_index(name='D. Enfermedad')

# Contar cuántas veces aparece cada legajo con código "v" (Vacaciones)
vacaciones = payroll[payroll['codigo'] == 'V'].groupby('legajo').size().reset_index(name='Vacaciones')

# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
feriado = payroll[payroll['codigo'] == 'DNL'].groupby('legajo').size().reset_index(name='Feriado')

# Contar cuántas veces aparece cada legajo con código "L" (Ausencias Injustificadas) || lPS, ART, LMED, -> Enfermedades
licencias = payroll[payroll['codigo'].isin(['LESP', 'LMED', 'MTM', 'LPS', 'CMh', 'LM', 'ART', 'LFF'])] \
          .groupby('legajo').size().reset_index(name='Licencia')

# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
mudanza = payroll[payroll['codigo'] == 'M'].groupby('legajo').size().reset_index(name='D. Mudanza')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
estudio = payroll[payroll['codigo'] == 'CE'].groupby('legajo').size().reset_index(name='D. Estudio')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
suspension = payroll[payroll['codigo'] == 'S'].groupby('legajo').size().reset_index(name='D. Suspension')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
tramite = payroll[payroll['codigo'] == 'DT'].groupby('legajo').size().reset_index(name='D. Tramite')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
falta_injustificada = payroll[payroll['codigo'] == 'X'].groupby('legajo').size().reset_index(name='Falta Inj.')



# Combinar todos los conteos en un solo DataFrame
conteos = enfermedad.merge(vacaciones, on='legajo', how='outer') \
                    .merge(feriado, on='legajo', how='outer') \
                    .merge(licencias, on='legajo', how='outer') \
                    .merge(mudanza, on='legajo', how='outer') \
                    .merge(estudio, on='legajo', how='outer') \
                    .merge(suspension, on='legajo', how='outer') \
                    .merge(tramite, on='legajo', how='outer') \
                    .merge(falta_injustificada, on='legajo', how='outer') 

nomina = nomina.merge(conteos, on='legajo', how='left')

# Completamos las columnas que se llenan a mano
for col in lista_columnas_vacias:
    nomina[col] = None

# Columna observaciones
nomina['OBSERVACIONES'] = nomina['legajo'].map(fechas_baja.set_index('legajo')['fecha_baja'])

# # Convertir OBSERVACIONES a datetime si no lo es
# nomina['OBSERVACIONES'] = pd.to_datetime(nomina['OBSERVACIONES'], errors='coerce')

# # Restar directamente el día de la fecha de baja, solo si existe
# nomina['Basico'] -= nomina['OBSERVACIONES'].dt.day.fillna(0).astype(int)

# Asegurar que la columna OBSERVACIONES es de tipo datetime
nomina['OBSERVACIONES'] = pd.to_datetime(nomina['OBSERVACIONES'], errors='coerce')

# Extraer el día como número entero, manejando NaN correctamente
nomina['dia_baja'] = nomina['OBSERVACIONES'].dt.day.fillna(0).astype(int)

nomina["dia_baja"] = nomina["dia_baja"].clip(upper=30)

# Restar solo cuando haya una fecha de baja registrada
nomina['Basico'] = nomina['Basico'] - nomina['dia_baja'].where(nomina['OBSERVACIONES'].notna(), 0)

# --------------------
# nomina['Basico'] = nomina['Basico'].where(nomina['Basico'] >= 30, 30 - nomina['Basico'])


# # Sumar todas las columnas que afectan al cálculo
# descuento = nomina[['Falta Inj.', 'D. Mudanza', 'D. Estudio', 'D. Enfermedad', 'Licencia', 'D. Tramite']].sum(axis=1)

# nomina["Basico"] = nomina['dia_baja'].where((nomina['Falta Inj.'].isna()) & (nomina["OBSERVACIONES"].notna()))

# # Restar solo cuando 'Falta Inj.' > 0, en caso contrario dejar 'Basico' sin cambios
# nomina['Basico'] -= descuento.where(nomina['Falta Inj.'] > 0, 0)



# nomina
# Asegurar que Basico no sea menor a 30
nomina['Basico'] = nomina['Basico'].where(nomina['Basico'] >= 30, 30 - nomina['Basico'])

# Sumar todas las columnas que afectan al cálculo
descuento = nomina[['Falta Inj.', 'D. Mudanza', 'D. Estudio', 'D. Enfermedad', 'Licencia', 'D. Tramite']].sum(axis=1)

# Aplicar descuento solo cuando 'Falta Inj.' > 0, dejando el resto sin cambios
nomina['Basico'] = nomina['Basico'] - descuento.where(nomina['Falta Inj.'] > 0, 0)

# Ajustar 'Basico' solo si 'Falta Inj.' es nulo y hay una observación
nomina.loc[(nomina['Falta Inj.'].isna()) & (nomina["OBSERVACIONES"].notna()), "Basico"] = nomina['dia_baja']
nomina


,legajo,documento,apellidos,nombres,Basico,D. Enfermedad,Vacaciones,Feriado,Licencia,D. Mudanza,...,D. Tramite,Falta Inj.,Hs Extras,Dep. Judicial,Prepaga,Enlazados,Ant. de Sueldos,Adic. Ventas,OBSERVACIONES,dia_baja
0,1317,18902640,CACERES PEREIRA,ANDRES FEDERICO,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
1,1600,19113210,RESTREPO ORDOSGOITIA,ARIANA ESTEFANNY,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
2,1601,19113447,RODRIGUEZ ALFONZO,NAIYELY HECNAY,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
3,884,19116652,APABLAZA GEREZ,ELIZABETH SABRINA,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
4,1229,23081765,PEREYRA,MARIA NORMA,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,2035,47194369,RIVEROS PINOS,GUADALUPE NICOL,30.0,15.0,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
598,1756,50665104,RABINO,CRISTIAN EMANUEL,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
599,2048,51709496,VILLEGAS BUGAO,MIREIA FLAVIA,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0
600,879,95105834,NUNES CARRERO,MARIANA CAROLINA,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,NaT,0


In [331]:
# exportamos a excel para analizar 
try:
    nomina = nomina.sort_values(by=['apellidos', 'nombres'], ascending=[True, True])
    nomina.to_excel(f"{file_name}", index=False)
    print(f"✅ Se extrajo correctamente el archivo {file_name}")
except Exception as error_exportacion:
    print(f"❌ Ocurrio un error al exportar el archivo\n \
          Error: {error_exportacion}")

❌ Ocurrio un error al exportar el archivo
           Error: [Errno 13] Permission denied: 'March 2025.xlsx'
